# Determining Population Densities and Intramicellar Distributions by Contrast Variation SANS

In [1]:
# Chi-Huan Tung
# National Tsing-Hua University
# Aug 2021
#
# Fit the I(Q) curve with soft ball P(Q) in J. Chem. Phys. 153, 184902 (2020)
# Calculate polymer distribution by contrast variation
#
# tensorflow 2.6.0

## Setup

Ensure you have installed these packages first  
``$ pip install tensorflow-probabilit``  
``$ pip install imageio``  
``$ pip install git+https://github.com/tensorflow/docs``  

In [2]:
from IPython import display

import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import tensorflow_probability as tfp
import time
import scipy.interpolate as interp
tf.config.run_functions_eagerly(True)

## Load data

In [3]:
def is_header(x):
    n_header = 5
    return x>=n_header

In [4]:
def get_struct_unpacker(fieldspecs, istart, iwidth):
    """
    Build the format string for struct.unpack to use, based on the fieldspecs.
    fieldspecs is a list of [name, start, width] arrays.
    Returns a string like "6s2s3s7x7s4x9s".
    """
    unpack_len = 0
    unpack_fmt = ""
    for fieldspec in fieldspecs:
        start = fieldspec[istart] - 1
        end = start + fieldspec[iwidth]
        if start > unpack_len:
            unpack_fmt += str(start - unpack_len) + "x"
        unpack_fmt += str(end - start) + "s"
        unpack_len = end
    struct_unpacker = struct.Struct(unpack_fmt).unpack_from
    return struct_unpacker

In [7]:
fieldspecs = [
    # Name, Start, Width, Type
    ["Q (1/A)", 8, 8, float],
    ["I(Q) (1/cm)", 26, 6, float],
    ["std. dev. I(Q) (1/cm)", 40, 8, float],
    ["sigmaQ", 55, 9, float],
    ["meanQ", 72, 8, float],
    ["ShadowFactor", 95, 1, int]
]

iname, istart, iwidth, itype = 0, 1, 2, 3  # field indexes

from operator import itemgetter
import struct
fieldspecs.sort(key=itemgetter(istart))
struct_unpacker = get_struct_unpacker(fieldspecs, istart, iwidth)
field_indices = range(len(fieldspecs))

In [14]:
data_path = '../data/S2/'
contrasts = np.array([100, 95, 90, 85, 80, 75])

for i, contrast in enumerate(contrasts):
    filename = data_path + 'S2_{:03d}.ABS'.format(contrast)
    with open(filename,'r') as fp:
        lines = fp.readlines()

    index_all = range(len(lines))
    index = list(filter(is_header, index_all))
    
    lines_data = list(itemgetter(*index)(lines))
    
    data_dict = {}
    for j, ld in enumerate(lines_data):
        raw_fields = struct_unpacker(ld.encode())
        ld_unpacked = {}
        for i in field_indices:
            fieldspec = fieldspecs[i]
            fieldname = fieldspec[iname]
            cast = fieldspec[itype]
            value = cast(raw_fields[i].decode().strip())
            ld_unpacked[fieldname] = [value]
        if j==0:
            data_dict = ld_unpacked
            continue
        data_dict = {key: [*data_dict[key], *ld_unpacked[key]] for key in data_dict}
    
    print('loaded '+ filename)

loaded ../data/S2/S2_100.ABS
loaded ../data/S2/S2_095.ABS
loaded ../data/S2/S2_090.ABS
loaded ../data/S2/S2_085.ABS
loaded ../data/S2/S2_080.ABS
loaded ../data/S2/S2_075.ABS
